## STEP 1 : 套件匯入

In [ ]:
from time import sleep        # 爬蟲禮貌，避免對方伺服器無法負荷請求而宕機
import requests               # 爬蟲請求
import json                   # JSON 資料型態
import pandas as pd           # 資料處理套件

## STEP 2 : 觀察網頁請求參數與響應格式

In [ ]:
# 藉由網站地址（URL）進行對方伺服器請求
# 嘗試進行請求，若響應狀態碼 200 即為成功
link = "https://www.lawplus.com.tw/rest/search/report?querySentence=%E8%AD%89%E5%88%B8%E8%A9%90%E6%AC%BA&keyword=%E8%AD%89%E5%88%B8%E8%A9%90%E6%AC%BA&prevKeyword=%E8%AD%89%E5%88%B8%E8%A9%90%E6%AC%BA&date=2020%2F01%2F01~2021%2F06%2F01&money=&sentence=&caseNum=&caseTypes=&courts=&levels=&jtypes=J&tags=&issue=&main=&judge=&judgeTypes=&lawyer=&litigant=&prosecutor=&clerk=&rows=10&page=2&sortField=&_=1624375988034"
res = requests.get(link)
print(res)

In [ ]:
# 觀察事項
## 1. 網頁請求響應的資料格式與內容
## 2. 觀察需要爬取內容（各別判決書的網頁）的方式（頁面切換等...）
json.loads(res.text)

## STEP 3 : 程式撰寫
##### 裁判家鏈接：https://www.lawplus.com.tw/#searchReport

In [ ]:
# 查詢結果的 URL
link = "https://www.lawplus.com.tw/rest/search/report?querySentence=%E8%AD%89%E5%88%B8%E8%A9%90%E6%AC%BA&keyword=&prevKeyword=%E8%AD%89%E5%88%B8%E8%A9%90%E6%AC%BA&date=2020%2F01%2F01~2021%2F06%2F01&money=&sentence=&caseNum=&caseTypes=&courts=&levels=&jtypes=J&tags=&issue=&main=&judge=&judgeTypes=&lawyer=&litigant=&prosecutor=&clerk=&rows=10&page={}&sortField=&_=1624376709444"

# 用於儲存個別判決書的鏈接
links = []

# page_no 代表每次迴圈該網頁頁碼
# range（起始頁碼，結束頁碼+1）
for page_no in range(1,10):
    # 於當下 page_no 的請求
    res = requests.get(link.format(page_no)).text
    # 將響應的內容格式轉換成 JSON 資料型態
    temp_data = json.loads(res)['rows']
    for each in temp_data:
        # 將個別判決書 URL 儲存至 links
        links.append(each['identifier'])
    # 休息 1 秒
    sleep(1)

In [ ]:
# 個別判決書的基本 URL
base_link = 'https://www.lawplus.com.tw/rest/search/report/{}'

# 用於儲存爬取的最終資料
data = []

# links 為儲存個別判決書的鏈接
for back_sec in links:
    # 於當下的判決書鏈接進行請求
    res_each = requests.get(base_link.format(back_sec)).text
    # 將響應的內容格式轉換成 JSON 資料型態
    temp_data_each = json.loads(res_each)
    data.append({
        'judge_date' : temp_data_each['response']['reportBase']['identifier'].split(',')[4],  # 裁判日期
        'issue' : temp_data_each['response']['reportBase']['issue'],            # 案由
        'type' : temp_data_each['response']['report']['type'],                  # 裁判類型
        'preside_judge' : temp_data_each['response']['report']['presideJudge'], # 審判長
        'puisne' : temp_data_each['response']['report']['puisne'],              # 陪審法官
        'appellant' : temp_data_each['response']['report']['appellant'],        # 上訴人
        'defendant' : temp_data_each['response']['report']['defendant'],        # 被告人
        'content' : temp_data_each['response']['reportBase']['content'],        # 內容
        'related_statute' : temp_data_each['response']['report']['statute']     # 相關法條
    })
    sleep(1)

## STEP 4 : 資料輸出

In [ ]:
df = pd.DataFrame(data)
df.to_csv('lawtech_0727.csv')

In [ ]:
df